In [75]:
using LinearAlgebra
using DifferentialEquations
using DynamicPolynomials
using TSSOS
using JuMP
using Random
using QuadGK
using NLopt


According to Section *State-transfer in linear networks* of [Kurt Jacobs *et al* 2016 EPL **114** 40007](https://iopscience.iop.org/article/10.1209/0295-5075/114/40007#epl17883s5), the equation of the motion for joint second moments
$$
    C = \langle {\bf v}^t {\bf v} \rangle 
    = \begin{pmatrix}
        \langle a^2 \rangle & \langle  a a^{\dagger} \rangle & \langle ab \rangle & \langle ab^{\dagger} \rangle \\
        \langle  a^\dagger a \rangle & \langle a^{\dagger} a^{\dagger} \rangle & \langle a^{\dagger} b \rangle & \langle a^{\dagger} b^{\dagger} \rangle \\
        \langle ab \rangle & \langle a^{\dagger} b \rangle & \langle b^2 \rangle & \langle b b^{\dagger} \rangle \\
        \langle a b^{\dagger} \rangle & \langle a^{\dagger} b^{\dagger} \rangle & \langle b^{\dagger} b \rangle & \langle b^{\dagger} b^{\dagger} \rangle 
     \end{pmatrix}
$$
is given by
$$
    \partial_t C = AC + CA^T.
$$

$$
    \partial_t vec(C) = (A ⊗ 1 + 1 ⊗ A) vec(C) 
$$

In [76]:
# the initial condition
vecC₀ = [0; 1; 0; 0;   2; 0; 0; 0;   0; 0; 0; 0;   0; 0; 1; 0];

In [77]:
# systems parameters
ω = 1.
g = 1.

function λ(t, x)
    # the polynomial shape for control
    sum(x[n] * t^(n - 1) for n = 1:length(x))
end


function A(t, x)
    # The matrix in the equation of motion for C
    λ₁ = λ(t, x)
    
    im * [ω    0      -λ₁*g  -λ₁*g
          0    -ω     λ₁*g  λ₁*g
         -λ₁*g -λ₁*g  ω      0
          λ₁*g λ₁*g   0     -ω]
end

A (generic function with 1 method)

## Utilities for working with Polynomials

In [78]:
function ∫(p::AbstractPolynomial, x::PolyVar, x_lower, x_upper)
    
    # get the index of the variable of integration
    ind_x = indexin([x], variables(p))[1]
        
    if isnothing(ind_x)
        # integration valuable is not found among vars
        return p * (x_upper - x_lower)
    end
    
    # get the indefinite integral
    int_p = sum(
        term * x * 1 // (exponents(term)[ind_x] + 1) for term in terms(p)
        init = 0 * x
    )
            
    # get the definite integral
    subs(int_p, x=>x_upper) - subs(int_p, x=>x_lower)
end

function ∫(M::AbstractMatrix, x::PolyVar, x_lower, x_upper)
   map(z -> ∫(z, x, x_lower, x_upper), M) 
end

function real_poly(p::Polynomial)
    #=
    Real part of the polynomial
    =#
    sum(
        real(c) * m for (c, m) in zip(coefficients(p), monomials(p))# if ~isapproxzero(abs(c))
    )
end

function square_frobenius_norm(M::AbstractArray)
    #=
    Square of the Frobenius norm of a matrix
    =#
    real_poly(sum(z' * z for z in M))
end

square_frobenius_norm (generic function with 1 method)

## Get the truncated Magnus expansion ($n=3$)

In [79]:
# final time
T = 1.

@polyvar x[1:5]
@polyvar t[1:3]

function 𝓐(t, x)
    #=
    The generator of motion entering the Magnus expansion
    =#
    a = A(t, x)
    
    𝓘 = Matrix(I, size(a))
    
    kron(a, 𝓘) + kron(𝓘, a)
end

function commutator(a, b)
    a * b - b * a
end 

# get the partial sum of the Magnus expansion
𝓐₁ = 𝓐(t[1], x)
𝓐₂ = 𝓐(t[2], x)

Ω = ∫(𝓐₁, t[1], 0, T);

# 2nd term in the Magnus expansion
Ω .+= 1//2 * ∫(∫(
    commutator(𝓐₁, 𝓐₂), 
    t[2], 0, t[1]), 
    t[1], 0, T
);

# 3nd term in the Magnus expansion

𝓐₃ = 𝓐(t[3], x)

Ω .+= 1//6 * ∫(∫(∫(
    commutator(𝓐₁, commutator(𝓐₂, 𝓐₃)) + commutator(commutator(𝓐₁, 𝓐₂), 𝓐₃),
    t[3], 0, t[2]),
    t[2], 0, t[1]),
    t[1], 0, T
);

## Chebyshve polynomial approximation for $\exp_p(\Omega^{(n)})$ with $n=3$ and $p=7$

In [80]:
using SpecialFunctions

"""
Chebyshev approximation for exp(Ω)
"""
function exp_chebyshev(Ω::AbstractMatrix, order::Integer)
    
    Tₙ₋₁ = I
    Tₙ  = Ω
    
    # The first two terms of Chebyshev series for exp
    series = besselj(0, 1) * Tₙ₋₁ + 2 * besselj(1, 1) * Tₙ
    
    for n=2:order
        Tₙ₊₁  = 2 * Ω * Tₙ + Tₙ₋₁
        
        series .+= 2 * besselj(n, 1) * Tₙ₊₁
        
        (Tₙ, Tₙ₋₁) = (Tₙ₊₁, Tₙ) 
    end
    
    series
end

Ω = convert(typeof(𝓐₁), Ω)

expΩ = exp_chebyshev(Ω, 7);

In [81]:
function local_minimize(obj::AbstractPolynomial, init_x::AbstractArray)
    #=
    Perform local minimization of obj polynomial using init_x as initial guess
    =#
    vars = variables(obj)

    @assert length(vars) == length(init_x)
    
    function g(a...)
        # Converting polynomial expression to function to be minimize
        obj(vars => a)
    end
    
    model = Model(NLopt.Optimizer)

    set_optimizer_attribute(model, "algorithm", :LD_MMA)

    set_silent(model)
    @variable(model, y[1:length(vars)])

    # set initial guess
    for (var, init_val) in zip(y, init_x)
        set_start_value(var, init_val)
    end

    register(model, :g, length(y), g; autodiff = true)
    @NLobjective(model, Min, g(y...))
    JuMP.optimize!(model)

    map(value, y)
end


function propagate(x::AbstractArray)   
    #=
    Solve the equation of motion
    =#
    
    rhs(C, p, t) = 𝓐(t, x) * C
    
    tspan = (0, T)
    
    DifferentialEquations.solve(
        ODEProblem(rhs, convert(Vector{ComplexF64}, vecC₀), tspan)
    )
end

function magnus_propagation(x::AbstractArray)   
    map(z -> z(x), expΩ) * vecC₀
end

function magnus_convergence_test(x::AbstractArray)
    val = quadgk(t -> opnorm(𝓐(t, x)), 0, T)[1] / π
    if val > 1
        println("Magnus expansion failed convergence test")
    end
    val
end

magnus_convergence_test (generic function with 1 method)

In [82]:
x_test = rand(size(x)[1])
sol_test = propagate(x_test)
magnus_convergence_test(x_test)

Magnus expansion failed convergence test


1.8051797175167237

In [83]:
sol_test.u[end][2]

3.2458501364949623 - 1.677218587838644e-16im

### Construct the objective function $\langle  a^\dagger a \rangle$ 

In [84]:
obj = (expΩ * vecC₀)[2]
obj = real_poly(obj);

In [88]:
# Get the global minimum via TSSOS library
#opt,sol,data = tssos_first(obj, variables(obj); QUIET = true, solution = true)
opt,sol,data = tssos_first(
    [obj; [ξ + 2 for ξ in x]; [-ξ + 2 for ξ in x]], 
    variables(obj), maxdegree(obj) ÷ 2, numeq=0; QUIET = true, solution = true
)

previous_sol = sol
previous_opt = opt
    
while ~isnothing(sol)
    previous_sol = sol
    previous_opt = opt
            
    opt,sol,data = tssos_higher!(data; QUIET = true, solution = true)
end
    
tssos_glob_obj_min = previous_opt
glob_min_x = previous_sol

***************************TSSOS***************************
TSSOS is launching...
optimum = 0.9789678933327952
Found a local optimal solution giving an upper bound: 0.9779393835617652 and a relative optimality gap: 0.0010285097710299729.


5-element Vector{Float64}:
  1.0784917609437985
 -2.9999994964037024
 -2.0308367366588125
  2.9999994883084127
  2.9999995893460194

In [89]:
propagate(glob_min_x).u[end][2]

0.9735254662497987 - 1.1419682707605287e-17im

In [90]:
magnus_convergence_test(glob_min_x)

Magnus expansion failed convergence test


1.0131852888380632

In [70]:
rand_loc_x = local_minimize(obj, rand(size(x)[1]))

809.2693330142467

In [71]:
magnus_convergence_test(rand_loc_x), magnus_convergence_test(glob_min_x)

(0.640956545300714, 0.7844658165654872)

real(propagate(glob_min_x).u[end][2]), sqrt(obj(glob_min_x)), real(magnus_propagation(glob_min_x)[2])

In [72]:
magnus_convergence_test(glob_min_x)

0.7844658165654872

In [ ]:
loc_min_x = local_minimize(obj, glob_min_x)
real(propagate(loc_min_x).u[end][2]), sqrt(obj(loc_min_x)), real(magnus_propagation(loc_min_x)[2])

In [ ]:
magnus_convergence_test(loc_min_x)

In [ ]:
rand_loc_x = local_minimize(obj, rand(size(glob_min_x)[1]))

real(propagate(rand_loc_x).u[end][2]), real(magnus_propagation(rand_loc_x)[2])

In [ ]:
magnus_convergence_test(rand_loc_x)

In [ ]:
rand_loc_x

In [ ]:
2*π/20